Dataset = https://www.kaggle.com/datasets/stackoverflow/stacksample

Licensed under https://creativecommons.org/licenses/by-sa/3.0/

For internal use only

In [1]:
import pandas as pd

### Extract all questions with the python tag

In [2]:
tags = pd.read_csv("/Users/marci/Downloads/archive/Tags.csv")

In [9]:
python_tags = tags[tags["Tag"] == "python"]
python_tags

,Id,Tag
312,11060,python
503,17250,python
546,19030,python
905,31340,python
1027,34020,python
...,...,...
3750788,40141710,python
3750886,40142380,python
3750921,40142600,python
3750940,40142840,python


In [13]:
questions = pd.read_csv("/Users/marci/Downloads/archive/Questions.csv", encoding="ISO-8859-1")

In [21]:
python_questions = questions[questions["Id"].isin(python_tags["Id"])]

### Only use question with score > 5

In [145]:
python_questions_filtered = python_questions[python_questions["Score"] > 5]

In [146]:
python_questions_filtered

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
113,11060,912.0,2008-08-14T13:59:21Z,NaN,18,How should I unit test a code-generator?,<p>This is a difficult and open-ended question...
184,17250,394.0,2008-08-20T00:16:40Z,NaN,24,Create an encrypted ZIP file in Python,<p>I'm creating an ZIP file with ZipFile in Py...
331,31340,242853.0,2008-08-27T23:44:47Z,NaN,71,"How do threads work in Python, and what are co...",<p>I've been trying to wrap my head around how...
370,34020,3561.0,2008-08-29T05:43:16Z,NaN,17,Are Python threads buggy?,<p>A reliable coder friend told me that Python...
377,34570,577.0,2008-08-29T16:10:41Z,2011-11-08T16:11:43Z,13,What is the best quick-read Python book out th...,<p>I am taking a class that requires Python. W...
...,...,...,...,...,...,...,...
1253188,39885770,3814425.0,2016-10-06T00:40:44Z,NaN,6,Most efficient way to determine overlapping ti...,<p>I am trying to determine what percentage of...
1256873,39971030,674039.0,2016-10-11T05:33:03Z,NaN,6,Make an object that behaves like a slice,<p>How can we make a class represent itself as...
1260719,40061280,1080804.0,2016-10-15T16:06:41Z,NaN,6,Why can yield be indexed?,<p>I thought I could make my python (2.7.10) c...
1262915,40111730,2907948.0,2016-10-18T15:01:30Z,NaN,7,How to use a dict to subset a DataFrame?,"<p>Say, I have given a DataFrame with most of ..."


### Match with answers

In [90]:
answers = pd.read_csv("/Users/marci/Downloads/archive/Answers.csv", encoding="ISO-8859-1")

In [147]:
python_answers_filtered = answers[answers["ParentId"].isin(python_questions_filtered["Id"])]

### Get answer with highest score

In [148]:
python_answers_filtered = python_answers_filtered.loc[python_answers_filtered.groupby(['ParentId'])['Score'].idxmax()]

### Extract ID and Body + rename body to answer

In [149]:
python_answers_filtered = python_answers_filtered[["ParentId", "Body"]]

In [150]:
python_answers_filtered = python_answers_filtered.rename(columns={"Body": "Answer"})

In [151]:
python_answers_filtered

,ParentId,Answer
412,11060,<p>I started writing up a summary of my experi...
877916,17250,<p>I created a simple library to create a pass...
1353,31340,"<p>Yes, because of the Global Interpreter Lock..."
1520,34020,<p>Python threads are good for <strong>concurr...
1547,34570,"<p>I loved <a href=""http://www.diveintopython...."
...,...,...
2007190,39885770,<p><strong><em>setup</em></strong><br>\ncreate...
2006542,39971030,<p>TLDR: It's impossible to make custom classe...
2010661,40061280,<p>You are not indexing. You are yielding a li...
2013032,40111730,<p>You can create a look up data frame from th...


In [152]:
python_q_a = pd.merge(python_questions_filtered, python_answers_filtered, left_on="Id", right_on="ParentId")

### Get rid of html tags

In [153]:
!pip install lxml
!pip install cssselect

In [154]:
import lxml.html

def get_text(html_string):
    page = lxml.html.document_fromstring(html_string)
    return page.cssselect('body')[0].text_content()

In [155]:
python_q_a["Body"] = python_q_a["Body"].apply(lambda x: get_text(x))
python_q_a["Answer"] = python_q_a["Answer"].apply(lambda x: get_text(x))

In [156]:
python_q_a

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,ParentId,Answer
0,11060,912.0,2008-08-14T13:59:21Z,NaN,18,How should I unit test a code-generator?,This is a difficult and open-ended question I ...,11060,I started writing up a summary of my experienc...
1,17250,394.0,2008-08-20T00:16:40Z,NaN,24,Create an encrypted ZIP file in Python,I'm creating an ZIP file with ZipFile in Pytho...,17250,I created a simple library to create a passwor...
2,31340,242853.0,2008-08-27T23:44:47Z,NaN,71,"How do threads work in Python, and what are co...",I've been trying to wrap my head around how th...,31340,"Yes, because of the Global Interpreter Lock (G..."
3,34020,3561.0,2008-08-29T05:43:16Z,NaN,17,Are Python threads buggy?,A reliable coder friend told me that Python's ...,34020,Python threads are good for concurrent I/O pro...
4,34570,577.0,2008-08-29T16:10:41Z,2011-11-08T16:11:43Z,13,What is the best quick-read Python book out th...,I am taking a class that requires Python. We w...,34570,"I loved Dive Into Python, especially if you're..."
...,...,...,...,...,...,...,...,...,...
4424,39885770,3814425.0,2016-10-06T00:40:44Z,NaN,6,Most efficient way to determine overlapping ti...,I am trying to determine what percentage of th...,39885770,setup\ncreate 2 time series\n\nfrom StringIO i...
4425,39971030,674039.0,2016-10-11T05:33:03Z,NaN,6,Make an object that behaves like a slice,How can we make a class represent itself as a ...,39971030,TLDR: It's impossible to make custom classes r...
4426,40061280,1080804.0,2016-10-15T16:06:41Z,NaN,6,Why can yield be indexed?,I thought I could make my python (2.7.10) code...,40061280,You are not indexing. You are yielding a list;...
4427,40111730,2907948.0,2016-10-18T15:01:30Z,NaN,7,How to use a dict to subset a DataFrame?,"Say, I have given a DataFrame with most of the...",40111730,You can create a look up data frame from the d...


In [157]:
idx = 80
print(python_q_a.iloc[idx]["Body"])
print(python_q_a.iloc[idx]["Answer"])

Is there a simple way to support wildcards ("*") when searching strings - without using RegEx?

Users are supposed to enter search terms using wildcards, but should not have to deal with the complexity of RegEx:

"foo*"   =>  str.startswith("foo")
"*foo"   =>  str.endswith("foo")
"*foo*"  =>  "foo" in str


(it gets more complicated when there are multiple search terms though, e.g. "foo*bar*baz")

This seems like a common issue, so I wonder whether there's a ready-made solution for it.

Any help would be greatly appreciated!

You could try the fnmatch module, it's got a shell-like wildcard syntax.



In [160]:
python_q_a.to_csv("python_qa.csv")